In [40]:
import Utils
from DataLoader import DataLoader
from FeatureCreator import FeatureCreator
from Model import Model
from CouponCreator import CouponCreator
import numpy as np
import pandas as pd
config = Utils.read_json('../config.json')
Utils.print_json(config)


dataloader = DataLoader(config)
dataset = dataloader.get_dataset()

{
    "data": {
        "path": "../data/",
        "files": {
            "baskets": "baskets.parquet",
            "coupons": "coupons.parquet",
            "coupons_index": "coupon_index.parquet"
        }
    },
    "model": {
        "test_week": 90,
        "n_shoppers": 2000,
        "n_products": 250,
        "train_window": 4,
        "trend_windows": [
            1,
            3,
            5
        ],
        "n_coupons": 5,
        "discounts": [
            0.15,
            0.2,
            0.25,
            0.3
        ]
    }
}
Read dataset.parquet.gzip from cache...
Successfully read dataset.parquet.gzip into memory.


In [28]:
dataset['week'].unique()

array([86, 87, 88, 89, 90], dtype=int64)

In [41]:
 
temp = pd.DataFrame()
#temp['product'] = np.arange(250)
elast = []

train_dataset = dataset[dataset['week'] < test_week]
for i in range(250):
    product             = train_dataset[train_dataset['product'] == i]
    
    reg_price_all       = product[product['discount'] == 0]
    discount_30_all     = product[product['discount'] == 0.3]
    
    
    reg_price_offer      = len(reg_price_all)
    discount_30_offer    = len(discount_30_all)
    
    reg_price_buy        = len(reg_price_all[reg_price_all['purchased'] == 1])
    discount_30_buy      = len(discount_30_all[discount_30_all['purchased'] == 1])

    
    reg_price_buy_rate   = reg_price_buy / reg_price_offer
    discount_buy_rate    = discount_30_buy / discount_30_offer
    
    elast.append((discount_buy_rate - reg_price_buy_rate) / (0.3 * reg_price_buy_rate))
      
    
temp['elasticities'] = elast
dataset = dataset.merge(temp, how='left', on='product')

temp['elasticities'].mean()

KeyError: 'product'

In [39]:
temp['elasticities']

0       4.012936
1      -0.309028
2       4.584342
3      -3.333333
4       9.203956
         ...    
245    -3.333333
246     4.138095
247    20.770891
248     0.791287
249     1.761613
Name: elasticities, Length: 250, dtype: float64

In [2]:
no_discount = dataset[dataset['discount'] == 0]
no_discount_buy = dataset[dataset['price']  > 0]

discount_30 = dataset[dataset['discount'] == 0.3]
discount_30_buy = dataset[dataset['price'] > 0]


no_discount_all = no_discount.groupby(['week'])['product'].apply(list).reset_index(name='no_discount_all')
no_discount_buy = no_discount.groupby(['week'])['product'].apply(list).reset_index(name='no_discount_buy')
discount_30_all = no_discount.groupby(['week'])['product'].apply(list).reset_index(name='discount_30_all')
discount_30_buy = no_discount.groupby(['week'])['product'].apply(list).reset_index(name='discount_30_buy')

dataset = dataset.merge(no_discount_all, how='left', on=(['week']))
dataset = dataset.merge(no_discount_buy, how='left', on=(['week']))
dataset = dataset.merge(discount_30_all, how='left', on=(['week']))
dataset = dataset.merge(discount_30_buy, how='left', on=(['week']))

In [5]:
no_discount_all
#no_discount_all = dataset['no_discount_all'][product].count()


,week,no_discount_all
0,86,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,87,"[0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,88,"[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,89,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
4,90,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."


In [76]:
DATALOADER
    
    '''
    Helper to derive price elasticities
    create list for 
    '''
    
no_discount = dataset[dataset['discount'] == 0]
no_discount_buy = dataset[dataset['price']  > 0]

discount_30 = dataset[dataset['discount'] == 0.3]
discount_30_buy = dataset[dataset['price'] > 0]


no_discount_all = no_discount.groupby(['week'])['product'].apply(list).reset_index(name='no_discount_all')
no_discount_buy = no_discount.groupby(['week'])['product'].apply(list).reset_index(name='no_discount_buy')
discount_30_all = no_discount.groupby(['week'])['product'].apply(list).reset_index(name='discount_30_all')
discount_30_buy = no_discount.groupby(['week'])['product'].apply(list).reset_index(name='discount_30_buy')

dataset = dataset.merge(no_discount_all, how='left', on=(['week']))
dataset = dataset.merge(no_discount_buy, how='left', on=(['week']))
dataset = dataset.merge(discount_30_all, how='left', on=(['week']))
dataset = dataset.merge(discount_30_buy, how='left', on=(['week']))



FEATURECREATOR

for product in dataset['product'].unique()
    no_discount_all = dataset['no_discount_all'][product].count()
    no_discount_buy = dataset['no_discount_buy'][product].count()
    discount_30_all = dataset['discount_30_all'][product].count()
    discount_30_buy = dataset['discount_30_buy'][product].count()
    
    reg_price_buy_rate = no_discount_buy / no_discount_all
    reg_price_offer = total_basket_count - all_discounts_offers

    
    product_elast = (discount_buy_rate - reg_price_buy_rate) / (0.3 * reg_price_buy_rate)

In [ ]:
DATALOADER

def elasticities_prep(dataset):
    '''
    Helper to derive price elasticities
    create list for 
    '''
    no_discount = baskets[baskets['discount'] == 0]
    no_discount_buy = baskets[baskets['price']  > 0]

    discount_30 = baskets[baskets['discount'] == 30]
    discount_30_buy = baskets[baskets['price'] > 0]


    no_discount = no_discount.groupby(['week'])['product'].apply(list).reset_index(name='no_discount')
    no_discount_buy = no_discount.groupby(['week'])['product'].apply(list).reset_index(name='no_discount_buy')
    discount_30 = no_discount.groupby(['week'])['product'].apply(list).reset_index(name='discount_30')
    discount_30_buy = no_discount.groupby(['week'])['product'].apply(list).reset_index(name='discount_30_buy')

    reg_price_offer = 100000 - discount_30


FEATURECREATOR

for product in baskets['products'].unique()
    discount_buy_rate = discount_30_buy / discount_30
    reg_price_buy_rate = no_discount_buy / no_discount
    elast = []
    temp = pd.DataFrame()
    temp['product'] = np.arange(250)
    temp['week'] = i + 1
    elast.append((discount_buy_rate - reg_price_buy_rate) / (0.3 * reg_price_buy_rate))

    
    
    
    
                reg_price_buy = len(reg_price[reg_price['discount'] == 0])
                all_discounts_offers = len(reg_price[reg_price['discount'] > 0])
                reg_price_offer = total_basket_count - all_discounts_offers
                reg_price_buy_rate = reg_price_buy / reg_price_offer
                discount_30 = reg_price[reg_price['discount'] == 30]
                discount_30_offer = len(discount_30)
                discount_30_buy = (discount_30['price'] != 0).sum()
                